# 결과물 구글 드라이브에 올려주세요

In [15]:
#%matplotlib inline

import os
import json

from pandas import Series
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from matplotlib import pyplot as plt

from tqdm.notebook import tqdm

# 액션 확정한 lda 파일(csv)불러오기
- topic_num에 액션 개수 입력

In [16]:
topic_num = 5 #액션 개수 입력

result_sentence = pd.read_csv('[최종]new_touchpoint1_actor2_식사챙김_문장분리재할당_원문.csv')
result_sentence.to_feather("[최종]new_touchpoint1_actor2_식사챙김_문장분리재할당_원문.ftr")
result_sentence = pd.read_feather("[최종]new_touchpoint1_actor2_식사챙김_문장분리재할당_원문.ftr", columns=None, use_threads=True)

result_sentence


,Unnamed: 0,original,sentence,topic
0,0,0,토일 외식+배달 총 얼마쓰세요?,1
1,1,0,감사합니당,5
2,3,0,배달 외식 거의 안해요.,1
3,4,0,22222하고싶은데 애들도 안 나가고 배달시키고 싶은데 맛있는곳도 없고....,2
4,7,0,배달은 5만원전후..외식은 10~15만원..5인기준..웬만하면 외식이든 배달이든..,1
...,...,...,...,...
43572,97030,13229,이런건 아니지 눈나 사랑해 가래떡맛있겟다 아래 어플이 뭐야 동운부,2
43573,97031,13230,다들 탕비실에 뭐있어 복지로 샐러드 놓으면 먹을거야,2
43574,97032,13230,총무인데 예산이 좀 늘어서 과자 말고도 주먹밥이나 샐러드 같은 간편식 주문하려고 하...,2
43575,97033,13230,근데 샐러드는 신선도가 생명이라 재고 관리가 힘들거 같은데 그런가 요즘 아침마다 마...,2


# 감성분석 (sentiment score)
- 1) 액션 순서대로 키워드 입력
- 2) 감성 점수 도출된것 확인 후 CAM에 입력

In [17]:
# 감성 사전으로 단어를 평가하는 함수
# token = 문자열 단어
def score_word(token):
    with open('SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
        data = json.load(f)
    result = ['None','None']
    for i in range(0, len(data)):
        if data[i]['word'] == token:
            result.pop()
            result.pop()
            result.append(data[i]['word_root'])
            result.append(data[i]['polarity'])
    return result[1]

In [18]:
# 문장을 평가하는 함수
# sentence = 문자열 문장
def score_sentence(sentence):
    okt=Okt()
    tokens = okt.pos(sentence, stem = True)

    token = []
    test = []
    for i in tokens:
        token.append(i[0])
        test.append(score_word(i[0]))

    prenegative = ["안", "못"]
    sunegative = ["않다", "모르다"]
    adjust = []

    for a, i in enumerate(test):

        try:
            if token[a - 1] in prenegative:
                adjust.append(int(i) * -1)
            else:
                adjust.append(i)
        except:
            adjust.append(i)

    adjust2 = []
    for a, i in enumerate(adjust):
        try:
            if token[a + 1] in sunegative:
                adjust2.append(int(i) * -1)
            else:
                adjust2.append(i)
        except:
            adjust2.append(i)

    while True:
        try:
            adjust2.remove("None")
        except:
            break

    test2 = []
    for i in adjust2:
        test2.append(int(i))

    try:
        test2[-1] = test2[-1] * 3
    except:
        pass

    try:
        return sum(test2)/len(test2)
    except:
        return 0

In [19]:
# 원문의 앞뒤 문장을 고려하여 어떤 키워드를 포함하는 문장을 감성분석하는 함수
# df = SNA 단계의 결과로 나온 텍스트 데이터
# keyword = 문자열 단어
def keyword_context_sentiment(df, keyword):

    target = []
    for keyword in keyword:
        for i in df["sentence"]:
            if keyword in i:
                target.append(i)
            else:
                pass

    target = list(set(target))
    print("키워드를 포함하는 문장 수: ", len(target))
    result = []
    dexx = []

    #if문 이유는 키워드를 포함하는 문장이 어떤 문단에 포함돼있냐에 따라 다른 점수가 나올 수 있음
    if len(target) == 0: # 키워드 포함하는 문장이 없는 경우
        dexx.append(keyword)
        result.append(0)

        final_result = {"index" : dexx, "sentiment" : result}
        final_result = pd.DataFrame(final_result)

    else: # 키워드 포함하는 문장이 하나일 경우
        for i in tqdm(target):

            gram = df[df["sentence"] == i]


            if len(gram) == 1: #하나만 나올 때는 그냥 계산
                dex = int(gram.index[0]) #인덱스 변수 설정

                if dex == df.index[0]: # 맨 앞 인덱스 #맨 앞 문단의 맨 앞 문장 경우 고려 [dex-1]이 정의되지 않아 오류 발생. 차단

                    if df.loc[dex]["original"] != df.loc[dex + 1]["original"]:
                        consider = df.loc[dex]["sentence"]
                    else:
                        consider = df.loc[[dex, dex + 1]]["sentence"]

                elif dex == df.index[-1]: #맨 뒤 인덱스 #맨 뒤 문단의 맨 뒤 문장 고려 [dex+1]이 정의되지 않아 오류 발생. 차단

                    if df.loc[dex - 1]["original"] != df.loc[dex]["original"]:
                        consider = df.loc[dex]["sentence"]
                    else:
                        consider = df.loc[[dex - 1, dex]]["sentence"]

                elif df.loc[dex - 1]["original"] != df.loc[dex]["original"]:
                    #해당 문장이 문단에 맨 앞인지 맨 뒤인지를 고려한 것
                    #해당 문장이 맨 앞 문장이면 인덱스가 앞으로 가면 문단 정보가 바뀜
                    if df.loc[dex]["original"] != df.loc[dex + 1]["original"]:
                        consider = df.loc[dex]["sentence"]
                    else:
                        consider = df.loc[[dex, dex + 1]]["sentence"]

                elif df.loc[dex]["original"] != df.loc[dex + 1]["original"]:
                    #해당 문장이 맨 뒤 문장이면 인덱스가 뒤로 가면 문단 정보가 바뀜
                    consider = df.loc[[dex - 1, dex]]["sentence"]

                else: # 위에 경우가 모두 아니면 해당 문장, 앞, 뒤를 모두 고려한 점수
                    consider = df.loc[[dex - 1, dex, dex + 1]]["sentence"]


                try:
                    consider = pd.DataFrame(consider) #1문단 1문장일 경우
                except:
                    consider = pd.DataFrame([consider]) #1문단 여러 문장일 경우 (문장이 중간에 있어서 앞뒤 계산이 되는 경우)

                consider.columns = ["sentence"]

                sentscore = 0
                for i in consider["sentence"]:
                    sentscore += score_sentence(i)
                result.append(sentscore/len(consider))  # 키워드가 포함된 문장 앞뒤 문장 각각의 문장 감성점수의 평균
                dexx.append(dex)


            else: # 키워드를 포함한 문장이 2개 이상일때 #리스트로 묶어서 동시에 고려
                for dex in list(gram.index):

                    if dex == df.index[0]: # 0번 인덱스
                        if df.loc[dex]["original"] != df.loc[dex + 1]["original"]:
                            consider = df.loc[dex]["sentence"]
                        else:
                            consider = df.loc[[dex, dex + 1]]["sentence"]

                    elif dex == df.index[-1]: #맨뒤 인덱스
                        if df.loc[dex - 1]["original"] != df.loc[dex]["original"]:
                            consider = df.loc[dex]["sentence"]
                        else:
                            consider = df.loc[[dex - 1, dex]]["sentence"]

                    elif df.loc[dex - 1]["original"] != df.loc[dex]["original"]:
                        if df.loc[dex]["original"] != df.loc[dex + 1]["original"]:
                            consider = df.loc[dex]["sentence"]
                        else:
                            consider = df.loc[[dex, dex + 1]]["sentence"]

                    elif df.loc[dex]["original"] != df.loc[dex + 1]["original"]:
                        consider = df.loc[[dex - 1, dex]]["sentence"]

                    else:
                        consider = df.loc[[dex - 1, dex, dex + 1]]["sentence"]

                    try:
                        consider = pd.DataFrame(consider)

                    except:
                        consider = pd.DataFrame([consider])

                    consider.columns = ["sentence"]

                    sentscore = 0

                    for i in consider["sentence"]:
                        sentscore += score_sentence(i)

                    result.append(sentscore/len(consider))
                    dexx.append(dex)

        final_result = {"index" : dexx, "sentiment" : result}
        final_result = pd.DataFrame(final_result)

    return final_result.sentiment.mean()


## [오래 걸림!] 액션 순서대로 키워드 입력
- 액션 개수에 맞게 리스트 입력
- 각 액션을 대표하는 키워드 2~3개 정도씩 넣어주세요
- 혹시 너무 오래걸릴 경우에는 키워드를 미리 한번 검색해보시고 확인 후에 변경해서 넣어주셔도 됩니다.

In [6]:
## 테스트입니다. 시간이 얼마나 걸리는지 확인해보는 용도로 사용해보세요
## 또는 해당 키워드가 문장에 있는지 없는지 확인할 수 있습니다.
# Action에 해당하는 Keyword를 정의

keywords = ['운동']
keyword_context_sentiment(result_sentence, keywords)

키워드를 포함하는 문장 수:  2154


  0%|          | 0/2154 [00:00<?, ?it/s]

[0.010s][warning][os,thread] Attempt to protect stack guard pages failed (0x0000000169564000-0x0000000169570000).
[0.010s][warning][os,thread] Attempt to deallocate stack guard pages failed.


KeyboardInterrupt: 

In [20]:
########### 액션 순서대로 키워드 리스트를 입력해주세요 ############

sentiment=[]


keywords_list = [
                    ['비싸다','물가','식비'], # action 1
                    ['도시락','배송'], # action 2
                    ['위장','숙취'], # action 3
                    ['점심','식단'], # action 4
                    ['운동','제로','프로'] # action 5
                ]


for keywords in keywords_list :
    action_sentiment = keyword_context_sentiment(result_sentence, keywords)
    sentiment.append(round(action_sentiment,4))

print('액션의 감성 점수',sentiment)

키워드를 포함하는 문장 수:  587


  0%|          | 0/587 [00:00<?, ?it/s]

키워드를 포함하는 문장 수:  2629


  0%|          | 0/2629 [00:00<?, ?it/s]

키워드를 포함하는 문장 수:  327


...

키워드를 포함하는 문장 수:  4493


  0%|          | 0/4493 [00:00<?, ?it/s]

키워드를 포함하는 문장 수:  4421


  0%|          | 0/4421 [00:00<?, ?it/s]

액션의 감성 점수 [-0.5122, 0.4118, 0.0948, 0.2211, 0.2792]


In [ ]:
# sentiment=[1.51,1.3322, -2.0952]

In [21]:
# 액션의 감성점수를 시그모이드 함수로 점수 바꿔주기
import math

def sigmoid(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

In [22]:
sent_score = []

for i in sentiment:
    sc = sigmoid(i)
    sent_score.append(round(sc*10,4))
print('★액션의 감성 점수(x10 해서 cam에 입력):', sent_score) #####  액션 순서입니다

★액션의 감성 점수(x10 해서 cam에 입력): [3.7468, 6.0152, 5.2368, 5.5505, 5.6935]


# 기회영역 구하기
- importance : imp_score
- satisfaction : sent_score
- opportunity : opp_score

## satisfaction
- sent_score = sactisfaction (tv프로젝트 참고)

## importance

### importance값 구하기

In [23]:
df = result_sentence.copy()
imp_score = []
for i in range(topic_num):
    ims = ((len(df.loc[df['topic']==i]))/len(df))
    imp_score.append(round(ims,4))

In [ ]:
# imp_score = []

# a1 = round(len(df.loc[df['topic']==2])/len(df),4)  # action 1 의 토픽 번호 입력
# a2 = round(len(df.loc[df['topic']==1])/len(df),4)  # action 2의 토픽 번호 입력
# a3 = round(len(df.loc[df['topic']==0])/len(df),4)  # action 3의 토픽 번호 입력
# #a4 = round(len(df.loc[df['topic']==3])/len(df),4)  # action 4의 토픽 번호 입력
# #a5 = round(len(df.loc[df['topic']==2])/len(df),4)  # action 5의 토픽 번호 입력

# imp_score.append(a1)
# imp_score.append(a2)
# imp_score.append(a3)
# #imp_score.append(a4)
# #imp_score.append(a5)

### standard scaler 적용 후 sigmoid 적용

In [24]:
from sklearn.preprocessing import StandardScaler

## 일단 df로 변환
imp_score_df = pd.DataFrame(imp_score, columns=['imp score'])

# Standardization 평균 0 / 분산 1
scaler = StandardScaler()
scaler = scaler.fit_transform(imp_score_df)


imp_score = []

for i in scaler:
    ims = sigmoid(i)
    imp_score.append(round(ims*10,4))
print('액션의 중요도 점수(importance score):', imp_score) #####  액션 순서입니다

액션의 중요도 점수(importance score): [2.4861, 3.7655, 8.2876, 3.3119, 6.7617]


## opportunity Score
- 𝑂𝑝𝑝𝑜𝑟𝑡𝑢𝑛𝑖𝑡𝑦=𝑖𝑚𝑝𝑜𝑟𝑡𝑎𝑛𝑐𝑒+𝑀𝑎𝑥(𝐼𝑚𝑝𝑜𝑟𝑡𝑎𝑛𝑐𝑒−𝑆𝑎𝑡𝑖𝑠𝑓𝑎𝑐𝑡𝑖𝑜𝑛, 0)

In [25]:
opp_score = []

for i in range(topic_num):
    opp = imp_score[i]+max(imp_score[i]-sent_score[i],0)
    opp_score.append(round(opp,4))
print('★액션의 기회영역 점수(x10 해서 CAM에 입력):',opp_score)

★액션의 기회영역 점수(x10 해서 CAM에 입력): [2.4861, 3.7655, 11.3384, 3.3119, 7.8299]


# 한번 더 정리(CAM에 입력할 숫자)

In [26]:
print('★액션의 감성 점수(x10 해서 cam에 입력):', sent_score) #####  액션 순서입니다
print('★액션의 기회영역 점수(x10 해서 CAM에 입력):',opp_score)

★액션의 감성 점수(x10 해서 cam에 입력): [3.7468, 6.0152, 5.2368, 5.5505, 5.6935]
★액션의 기회영역 점수(x10 해서 CAM에 입력): [2.4861, 3.7655, 11.3384, 3.3119, 7.8299]


# 저장
- file_name 바꿔주세요

In [28]:
file_name='Touchpoint1_actor2' # 파일 이름 변경!

name = [f'{file_name}_action{x}' for x in range(1,topic_num+1)] #액터 번호 입력

result = pd.DataFrame({'action': name,'sent score':sent_score, 'imp score':imp_score, 'opp score':opp_score })
result.to_csv(f'{file_name}_insight.csv',encoding='utf-8-sig') # actor번호
result

,action,sent score,imp score,opp score
0,Touchpoint1_actor2_action1,3.7468,2.4861,2.4861
1,Touchpoint1_actor2_action2,6.0152,3.7655,3.7655
2,Touchpoint1_actor2_action3,5.2368,8.2876,11.3384
3,Touchpoint1_actor2_action4,5.5505,3.3119,3.3119
4,Touchpoint1_actor2_action5,5.6935,6.7617,7.8299


# --------------------------- 여기까지----------------------------------